In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate();


In [ ]:
words = sc.textFile("/content/drive/MyDrive/CISC825/pollution.csv").flatMap(lambda line: line.split("\n"))
words.take(5)

['ozone,particullate_matter,carbon_monoxide,sulfure_dioxide,nitrogen_dioxide,longitude,latitude,timestamp',
 '103,81,33,31,40,10.116589665412905,56.22579478256016,2014-08-01 00:05:00',
 '99,79,35,36,44,10.116589665412905,56.22579478256016,2014-08-01 00:10:00',
 '102,80,33,36,47,10.116589665412905,56.22579478256016,2014-08-01 00:15:00',
 '107,82,33,36,45,10.116589665412905,56.22579478256016,2014-08-01 00:20:00']

In [ ]:
def RecordMapper(record):
  records = record.split(",")
  if records[0]=="ozone":
    return ("", (0, 0, 0, 0, 0, 0))
  else:
    return (records[7], (int(records[0]), int(records[1]), int(records[2]), int(records[3]), int(records[4]),  1))


In [ ]:
record_list_rdd = words.map(lambda x: RecordMapper(x))

In [ ]:
record_list_rdd.collect()
record_list_rdd.take(5)

[('', (0, 0, 0, 0, 0, 0)),
 ('2014-08-01 00:05:00', (103, 81, 33, 31, 40, 1)),
 ('2014-08-01 00:10:00', (99, 79, 35, 36, 44, 1)),
 ('2014-08-01 00:15:00', (102, 80, 33, 36, 47, 1)),
 ('2014-08-01 00:20:00', (107, 82, 33, 36, 45, 1))]

In [ ]:
record_counts_rdd = record_list_rdd.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2], x[3] + y[3], x[4] + y[4], x[5] + y[5]))
record_counts_rdd

PythonRDD[39] at RDD at PythonRDD.scala:53

In [ ]:
record_counts_rdd.take(5)

[('', (0, 0, 0, 0, 0, 0)),
 ('2014-08-01 01:55:00', (28008, 27625, 28509, 28256, 28061, 448)),
 ('2014-08-01 03:20:00', (28436, 28015, 29258, 28831, 28410, 448)),
 ('2014-08-01 04:25:00', (28200, 28823, 29752, 29337, 28650, 448)),
 ('2014-08-01 07:10:00', (29134, 28852, 30048, 30357, 29389, 448))]

In [ ]:
record_sum = record_counts_rdd.collect()
record_sum[:5]

[('', (0, 0, 0, 0, 0, 0)),
 ('2014-08-01 01:55:00', (28008, 27625, 28509, 28256, 28061, 448)),
 ('2014-08-01 03:20:00', (28436, 28015, 29258, 28831, 28410, 448)),
 ('2014-08-01 04:25:00', (28200, 28823, 29752, 29337, 28650, 448)),
 ('2014-08-01 07:10:00', (29134, 28852, 30048, 30357, 29389, 448))]

In [ ]:
output = "/content/drive/MyDrive/CISC825/records.csv"
with open(output, 'w', encoding="utf-8") as outputf:
  for r in record_sum[1:]:
    time = r[0]
    result = r[1]
    count = result[5]
    outputf.write(time+','+str(int(result[0]/count))+','+str(int(result[1]/count))+','+str(int(result[2]/count))+','+str(int(result[3]/count))+','+str(int(result[4]/count))+'\n')
    outputf.flush()